# Imports

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from tqdm import tqdm
import csv

# Loading and Splitting the data

Normalizing also

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and transforms
data_dir = "/kaggle/input/food41/images"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to EfficientNet-B7's expected size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset into train, validation, and test
indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(
    indices, test_size=0.2, random_state=42, stratify=dataset.targets
)
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

val_ratio = 0.5  # 50% of the test set will be used for validation
val_size = int(len(test_dataset) * val_ratio)
test_size = len(test_dataset) - val_size
val_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8)

print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")
print(f"Testing images: {len(test_dataset)}")

Training images: 80800
Validation images: 10100
Testing images: 10100


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Model Training

In [5]:
model = models.efficientnet_b4(pretrained=True)
num_classes = len(dataset.classes)  # Number of classes in the dataset
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)  # Modify classifier
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 198MB/s]


In [6]:
csv_file = "training_metrics.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Val Loss", "Val Accuracy"])

epochs = 1
best_val_loss = float("inf")
best_model_path = "best_model.pth"

for epoch in range(epochs):
    model.train()
    train_loss, train_correct = 0.0, 0

    # Training phase
    with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}") as pbar:
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

            pbar.set_postfix(loss=train_loss / len(train_loader))

    train_loss /= len(train_loader)
    train_accuracy = train_correct / len(train_dataset)

    # Validation phase
    model.eval()
    val_loss, val_correct = 0.0, 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = val_correct / len(val_dataset)

    with open(csv_file, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([epoch + 1, train_loss, train_accuracy, val_loss, val_accuracy])

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            "model": model,
            "classes": dataset.classes
        }, best_model_path)
        print(f"New best model saved with validation loss: {best_val_loss:.4f}")

    print(f"Epoch {epoch + 1}/{epochs} -> Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

Epoch 1/1: 100%|██████████| 2525/2525 [13:05<00:00,  3.22it/s, loss=2.81]  


New best model saved with validation loss: 1.4795
Epoch 1/1 -> Train Loss: 2.8082, Train Acc: 0.3464, Val Loss: 1.4795, Val Acc: 0.6206


# Loading the model and evaluating on the test set

In [7]:
checkpoint = torch.load(best_model_path, map_location=device)
model = checkpoint["model"]
class_names = checkpoint["classes"]
model = model.to(device)
model.eval()


def evaluate_model(loader):
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

test_preds, test_labels = evaluate_model(test_loader)

print("Test Classification Report:")
print(classification_report(test_labels, test_preds, target_names=class_names))
test_accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

/tmp/ipykernel_23/447767184.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(best_model_path, map_location=device)


Test Classification Report:
                         precision    recall  f1-score   support

              apple_pie       0.40      0.29      0.34       100
         baby_back_ribs       0.55      0.72      0.62        92
                baklava       0.68      0.74      0.71       108
         beef_carpaccio       0.62      0.62      0.62       104
           beef_tartare       0.62      0.53      0.57        94
             beet_salad       0.46      0.44      0.45        98
               beignets       0.81      0.78      0.79       107
               bibimbap       0.68      0.85      0.75       101
          bread_pudding       0.47      0.19      0.27        90
      breakfast_burrito       0.60      0.61      0.61        96
             bruschetta       0.55      0.48      0.51       107
           caesar_salad       0.70      0.69      0.70        98
                cannoli       0.66      0.72      0.69       103
          caprese_salad       0.48      0.65      0.55       